<a href="https://colab.research.google.com/github/rkpradhan/rkpradhan.github.io/blob/master/galahdr3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install vaex
#!pip install --upgrade ipython

In [ ]:
import vaex

In [ ]:
#!pip install galpy 

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd


In [ ]:
from astropy.io import ascii 
from astropy.table import Table, vstack, hstack,  join
import astropy.units as u
import astropy.coordinates as coord 


In [ ]:
# Matplotlib packages
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
params = {
    'font.family'        : 'sans',
    'font.size'          : 17,
    'axes.labelsize'     : 20,
    'ytick.labelsize'    : 16,
    'xtick.labelsize'    : 16,
    'legend.fontsize'    : 20,
    'text.usetex'        : False, 
    'text.latex.preamble': [r'\usepackage{upgreek}', r'\usepackage{amsmath}'],
    }   
plt.rcParams.update(params)

In [ ]:
import matplotlib.pyplot 
matplotlib.rcParams['text.usetex']=False
matplotlib.rcParams['text.latex.unicode']=False 

In [ ]:
#!wget https://datacentral.org.au/teamdata/GALAH/public/GALAH_DR3/GALAH_DR3_VAC_dynamics_v1.fits


In [ ]:
#!wget https://datacentral.org.au/teamdata/GALAH/public/GALAH_DR3/GALAH_DR3_main_allspec_v1.fits

In [ ]:
#df = Table.read("/content/GALAH_DR3_VAC_dynamics_v1.fits")

In [ ]:
!ls 

In [ ]:
# df1 = Table.read('/content/GALAH_DR3_main_allspec_v1.fits')


In [ ]:

# from astropy.table import Table, vstack
# tbl = Table.read("/content/GALAH_DR3_VAC_dynamics_v1.fits")
# try:
#     tbl2 = Table.read('/content/GALAH_DR3_main_allspec_v1.fits')
# except:
#     pass
# else:
#     tbl = join(tbl2, tbl, keys = 'sobject_id', join_type='inner', uniq_col_name= '{teff, logg}_{df1}', metadata_conflicts='silent')



In [ ]:
tbl2 = Table.read('/content/GALAH_DR3_main_allspec_v1.fits')


In [ ]:
para = tbl2['parallax']>0.0
tbl2 = tbl2[para]


In [ ]:
nr_galah_dr3_stars = np.where(
        np.isfinite(tbl2['ra']) &
        np.isfinite(tbl2['dec']) &
        np.isfinite(tbl2['pmra']) &
        np.isfinite(tbl2['pmdec']) &
        np.isfinite(tbl2['ra_error']) &
        np.isfinite(tbl2['dec_error']) &
        np.isfinite(tbl2['pmra_error']) &
        np.isfinite(tbl2['pmdec_error']) &
        (
            (
            np.isfinite(tbl2['rv_galah']) &
            np.isfinite(tbl2['e_rv_galah'])
            ) |
            (
            np.isfinite(tbl2['rv_gaia']) &
            np.isfinite(tbl2['e_rv_gaia'])
            )
        )
    )[0]

    # This should only me activated for tests with subsets of GALAH DR3
    #nr_galah_stars_dynamics = nr_galah_stars_dynamics[:100]

galah_gaia = tbl2[nr_galah_dr3_stars]
nr_stars = len(galah_gaia['sobject_id'])
print("Selected number of stars")

print(nr_stars)

In [ ]:

#galah_gaiam = galah_gaia[(galah_gaia['logg']>3.2) & (galah_gaia['logg']<4.1) &(galah_gaia['teff']>5000) & (galah_gaia['teff']< 7000)]
tbl2m = galah_gaia[(galah_gaia['logg']<3.5)& (galah_gaia['teff']>4000) &(galah_gaia['teff']<4600 )|(galah_gaia['logg']<4.)& (galah_gaia['teff']>4600) &(galah_gaia['teff']<5600 )]

tbl2m[:]

In [ ]:
#dist = 1/tbl2['parallax']
dist  = coord.Distance(parallax = u.Quantity(tbl2m['parallax']))
dist.max()

In [ ]:
# from astropy.coordinates import CartesianDifferential
# cs = coord.SkyCoord(ra = tbl2m['ra'], 
#                     dec = tbl2m['dec'],
#                     distance = dist, 
#                     pm_ra_cosdec = tbl2m['pmra'], 
#                     pm_dec = tbl2m['pmdec'], 
#                     radial_velocity = tbl2m['rv_galah'], 
#                     galcen_distance=8.*u.kpc,z_sun=15.*u.pc,
#                 galcen_v_sun=CartesianDifferential([10.0,235.,7.]*u.km/u.s))



In [ ]:
ra = np.array(tbl2m['ra'])
dec = np.array(tbl2m['dec'])
distance = np.array(dist*u.pc.to(u.kpc))
pmra = np.array(tbl2m['pmra'] )
pmdec = np.array(tbl2m['pmdec'] )
radial_velocity = np.array(tbl2m['rv_galah'])

In [ ]:
#!pip install galpy


In [ ]:
import galpy
#from galpy.potential import MWPotential2014 as pot
from galpy.potential.mwpotentials import McMillan17 as pot
from galpy.util.bovy_conversion import get_physical
from galpy.actionAngle import actionAngleStaeckel
from galpy.orbit import Orbit

# Reference values
#r_galactic_centre = 8.178*u.kpc # Gravity Collaboration, 2019, A&A, 625, 10
r_galactic_centre = 8.21*u.kpc # McMillan Potential, 2017
z_galactic_plane = 25.0*u.pc # Bland-Hawthorn & Gerhard, 2016, ARA&A, 54, 529

print('Reference frame:')
print('R_GC = '+str(r_galactic_centre)+' (McMillan, 2017, MNRAS, 465, 76)')
print('phi_GC = '+str(0*u.rad))
print('z_GC = '+str(z_galactic_plane)+' (Bland-Hawthorn & Gerhard, 2016, ARA&A, 54, 529)')

v_total_sun = (np.tan(6.379*u.mas)*r_galactic_centre/u.yr).to(u.km/u.s) # pm_l by Reid & Brunthaler 2004, ApJ, 616, 872
print('V_total_sun: = '+"{:.2f}".format(v_total_sun)+' (Reid & Brunthaler 2004, ApJ, 616, 872)')
v_peculiar = [11.1, 15.17, 7.25]*u.km/u.s # U and W from Schoenrich, Binney, Dehnen, 2010, MNRAS, 403, 1829, V so that V = V_total-V_sun
print('V_peculiar = ',(v_peculiar),' (U and W from Schoenrich, Binney, Dehnen, 2010, MNRAS, 403, 1829)')
print('V-component of V_peculiar = 15.17 km/s, instead of 12.24 km/s by Schoenrich et al. (2010), for matching v_circular')
v_circular = np.round(v_total_sun-v_peculiar[1],1)
print('V_circular = ',(v_circular),' (McMillan, 2017, MNRAS, 465, 76)')

aAS = actionAngleStaeckel(
        pot   = pot,        #potential                                                                                                                                                                      
        delta = 0.45,       #focal length of confocal coordinate system                                                                                                                            
        c     = True        #use C code (for speed)                                                                                                                                                         
        )

#(RA = 17:45:37.224 h:m:s, Dec = −28:56:10.23 deg) (Reid& Brunthaler 2004)

In [ ]:
o = Orbit(
        #ra, dec, dist, pm_ra, pm_dec, v_los
        vxvv=[ra*u.deg,dec*u.deg,distance*u.kpc,pmra*u.mas/u.yr, pmdec*u.mas/u.yr,radial_velocity*u.km/u.s],
        ro=r_galactic_centre,
        vo=v_circular,
        zo=z_galactic_plane,
        solarmotion=[-11.1, 15.17, 7.25]*u.km/u.s,
        #solarmotion='schoenrich',
        radec=True
    )

In [ ]:
o.dist().max()

In [ ]:
from matplotlib import rc
rc('text', usetex = False)

In [ ]:
plt.plot(o.x(), o.z(), '.')

In [ ]:
#E = o.E(pot=pot,ro=r_galactic_centre,vo=v_circular,zo=z_galactic_plane)

In [ ]:
tbl2m['E'] = o.E(pot=pot,ro=r_galactic_centre,vo=v_circular,zo=z_galactic_plane)

In [ ]:
tbl2m['E_z'] = o.Ez(pot=pot,ro=r_galactic_centre,vo=v_circular,zo=z_galactic_plane)

In [ ]:
x,y,z, vx, vy, vz  = o.x(),o.y(), o.z(), o.vx(), o.vy(), o.vz()
tbl2m['U'] = o.U()
tbl2m['V'] = o.V()
tbl2m['W'] = o.W()

tbl2m['Lz'] = o.Lz()
tbl2m['L'] = o.L()
tbl2m['vlos']= o.vlos()
tbl2m['vphi'] = o.vphi()
tbl2m['vR'] = o.vR()
tbl2m['vT'] = o.vT()


In [ ]:
np.median(x)

In [ ]:
tbl2m['x'], tbl2m['y'], tbl2m['z'], tbl2m['vx'], tbl2m['vy'], tbl2m['vz'] = x,y, z, vx, vy, vz



In [ ]:
tbl2m

In [ ]:
# E = tbl2m['E']
# Lz = tbl2m['Lz']
# plt.plot(E, Lz, '.')
# plt.xlim(-464540078, 9464540078)
# plt.ylim(-2000000, 2000000)

#### convert into gc 

In [ ]:
# galcen = cs.transform_to(coord.Galactocentric(z_sun=0*u.pc,
#                                              galcen_distance=8.1*u.kpc))

In [ ]:
# x, y, z, vx, vy, vz = galcen.x, galcen.y, galcen.z, galcen.v_x, galcen.v_y, galcen.v_z

In [ ]:
#!pip install latex

In [ ]:
#!sudo apt install texlive texlive-latex-extra texlive-fonts-recommended dvipng

In [ ]:
from matplotlib import rc 
rc('text', usetex = False)

In [ ]:
tbl = vaex.from_astropy_table(tbl2m)

In [ ]:
tbl.export_hdf5('/content/output_data_galah.hdf5')


In [ ]:
df_output_galah =vaex.open('/content/output_data_galah.hdf5')


In [ ]:
#df_output_galah.export_hdf5('/content/output_data_galah.csv')


In [ ]:
df_vaex_astro= df_output_galah.to_astropy_table()
df_vaex_astro[:4]

In [ ]:
xcut_pos = df_output_galah.x>25
xcut_neg = df_output_galah.x<-25
xcut = (~xcut_pos)&(~xcut_neg)
dfhalo_xcut  = df_output_galah[xcut]


In [ ]:
ycut_pos = dfhalo_xcut.y>25
ycut_neg = dfhalo_xcut.y<-25
ycut = (~ycut_pos)&(~ycut_neg)
dfhalo_xycut =dfhalo_xcut[ycut]


In [ ]:
zcut_pos = dfhalo_xycut.z>25
zcut_neg = dfhalo_xycut.z<-25
zcut = (~zcut_pos)&(~zcut_neg)
dfhalo_xyzcut_galah= dfhalo_xycut[zcut]
dfhalo_xyzcut_galah

In [ ]:
dfhalo_xyzcut_galah.export_hdf5('/content/df_gaia2_galah3.hdf5')

In [ ]:
df_output_galah3 = vaex.open('/content/df_gaia2_galah3.hdf5')

In [ ]:
df_galah_gaia = df_output_galah3.to_astropy_table()

In [ ]:
from matplotlib import rc
rc('text', usetex = False)

In [ ]:
# import matplotlib.pyplot as plt 
# plt.figure(figsize=(10,5))
# ax1 = plt.subplot(1,2,1)
# ax1.plot(dfhalo_xyzcut_galah['x'], dfhalo_xyzcut_galah['z'], '.')
# ax1.set_title('z vs x')

# ax2 = plt.subplot(1,2,2)
# ax2.plot(dfhalo_xyzcut_galah['x'], dfhalo_xyzcut_galah['y'], '.')
# ax2.set_title('y vs x')

In [ ]:

df_output_galah3.plot(df_output_galah3.x, df_output_galah3.y, what=np.log(vaex.stat.count()+1), limits='99.7%');

In [ ]:
df_output_galah3.plot(df_output_galah3.x, df_output_galah3.z, what=np.log(vaex.stat.count()+1), limits='99.7%');

In [ ]:
df_output_galah3.plot([["x", "y"], ["x", "z"]], limits='99.7%',
        title="Face on and edge on", figsize=(10,4));

In [ ]:
df_output_galah3.plot([["x", "y"], ["x", "z"]],
        limits='99.7%',
        what=[np.log(vaex.stat.count()+1), vaex.stat.mean(df_output_galah3.E)],
        selection=[None, df_output_galah3.x < df_output_galah3.y],
        title="Face on and edge on", figsize=(10,10));

In [ ]:
df_output_galah3.plot([["x", "y"], ["x", "z"]],
        limits='99.7%',
        what=vaex.stat.mean(df_output_galah3.E),
        selection=[None, df_output_galah3.Lz < 0],
        visual=dict(column='selection'),
        title="Face on and edge on", figsize=(10,10));

In [ ]:
z = df_output_galah3.fe_h

In [ ]:
df_output_galah3.plot("Lz", "E",
        limits='99.7%',
        z="fe_h:-3.0,-1,8", show=True, visual=dict(row="z"),
        figsize=(12,8), f="log", wrap_columns=3);



In [ ]:
x, y, z, vx, vy, vz = df_galah_gaia.x, df_galah_gaia.y, df_galah_gaia.z, df_galah_gaia.vx, df_galah_gaia.vy, df_galah_gaia.vz

In [ ]:
# df_galah_gaia.plot([["vx", "vy"], ["vx", "vz"]], limits='95.0%',
#         title="Face on and edge on", figsize=(10,4));

In [ ]:
# !pip install vaex
# !pip install --upgrade ipython

In [ ]:
# vaex_df= vaex.from_astropy_table(tbl2m)

In [ ]:
# counts_x = vaex_df.count(binby=vaex_df.x, limits=[-10, 10], shape=64)
# counts_x

In [ ]:
# xycounts = vaex_df.count(binby=[vaex_df.x, vaex_df.y], limits=[[-10, 10], [-10, 20]], shape=(64, 128))
# xycounts

In [ ]:

# df = vaex_df(vaex_df.evaluate(vaex_df.x<0, selection=True))
# df.row

In [ ]:
# vaex_df_neg  = vaex_df[vaex_df.evaluate(vaex_df.x<0., selection=True)]

In [ ]:
# ra=tbl2m['ra']
# dec=tbl2m ['dec']
# pmra=tbl2m['pmra']
# pmdec=tbl2m ['pmdec']
# pmra_error=tbl2m['pmra_error']
# pmdec_error=tbl2m['pmdec_error']
# X=tbl2m['X_XYZ']
# Y =tbl2m['Y_XYZ']
# Z =tbl2m['Z_XYZ']
# U =tbl2m['U_UVW']
# V=tbl2m['V_UVW']
# W=tbl2m['W_UVW']
# R=tbl2m['R_Rzphi']
# z=tbl2m['z_Rzphi']
# phi=tbl2m['phi_Rzphi'] 
# vR =tbl2m['vR_Rzphi']
# vz=tbl2m['vz_Rzphi']
# vphi=tbl2m ['vphi_Rzphi']
# vT =tbl2m['vT_Rzphi']
# J_R =tbl2m['J_R']
# L_Z=tbl2m['L_Z']
# J_Z=tbl2m['J_Z']
# E=tbl2m['Energy']
# sobject = tbl2m['sobject_id']
# fe_h = tbl2m['fe_h']

In [ ]:

#data = {'ra': ra, 'dec': dec,   'pmra':pmra,  'pmdec':pmdec, 'X': X, 'Y': Y, 'Z': Z, 'U': U, 'V': V, 'W': W, 'R':R, 'z': z,'phi': phi, 'vR': vR, 'vz': vz, 'vphi': vphi, 'vT': vT, 'J_R': J_R, 'L_Z' : L_Z, 'J_Z':J_Z, 'Energy': E, 'fe_h' : fe_h}

In [ ]:
#df = pd.DataFrame(data = data)

In [ ]:
# import numpy as np
# r = np.sqrt(tblll['x']**2+tblll['y']**2+tblll['z']**2)
# tblll['r'] = r 

In [ ]:
import numpy as np
import matplotlib
matplotlib.rcParams['text.usetex'] = True
import matplotlib.pyplot as plt


In [ ]:
df_output_galah3

In [ ]:
def cx2sp( *args ):
    """
        cx2sp
    PURPOSE
        Converts cartesian (X,V) into spherical polar coordinates 
                                 (r, theta, phi, vr, vtheta, vphi). 
    
    CALLING SEQUENCE:
        r, theta, phi, vr, vtheta, vphi = 
            cx2sp(x, y, z, vx, vy, vz)
    INPUTS:
       x, y, z - Input cartesian position in kpc, scalar or an array  
       vx, vy, vz  - Input velocity components in km/s
       
    OUTPUTS:
       theta, phi - Output position angles in radians
              theta -> [0,2PI]; phi -> [-PI/2,PI/2.]
       r    - Output distance in kpc; [0,Inf]

       vr, vtheta, vphi   - Output velocities in km/s [-Inf,Inf]

    INPUT KEYWORD:
       
    REVISION HISTORY:

    REFERENCE: 
       phi is an azimuthal angle Right Handed coordinate system. 
       theta is an altitude angle, made by position vector with the positive z-axis
    """
    import numpy as np

    if len(args)==6:
        dim = 6
        x,y,z,vx,vy,vz = args
        x = np.atleast_1d(x)
        y = np.atleast_1d(y)
        z = np.atleast_1d(z)
        vx = np.atleast_1d(vx)
        vy = np.atleast_1d(vy)
        vz = np.atleast_1d(vz)

    else:
        dim = 3
        x,y,z = args

        x = np.atleast_1d(x)
        y = np.atleast_1d(y)
        z = np.atleast_1d(z)

    r = (x*x +y*y +z*z)**0.5
    arcos   = np.arccos; artan = np.arctan2
    theta   = arcos(z/r)
    phi     = artan(y,x)
    sinthi  = np.sin(theta); costhi = np.cos(theta); sinphi = np.sin(phi)
    cosphi  = np.cos(phi) 

    if dim==6:
        vr      = sinthi*cosphi*vx +sinthi*sinphi*vy +costhi*vz 
        vtheta  = costhi*cosphi*vx +costhi*sinphi*vy -sinthi*vz
        vphi    = -sinphi*vx +cosphi*vy
    
        return r, theta, phi, vr, vtheta, vphi
    return r, theta, phi

In [ ]:
x, y, z, vx, vy, vz = df_galah_gaia['x'], df_galah_gaia['y'], df_galah_gaia['z'], df_galah_gaia['vx'], df_galah_gaia['vy'], df_galah_gaia['vz'] 

#from vaex table 
#x, y, z, vx, vy, vz = df_galah_gaia.x, df_galah_gaia.y, df_galah_gaia.z, df_galah_gaia.vx, df_galah_gaia.vy, df_galah_gaia.vz 



In [ ]:
cxtosp = cx2sp(x, y, z, vx, vy, vz)

In [ ]:
#for astropy table
df_galah_gaia['r'],df_galah_gaia['theta'],df_galah_gaia['phi'],df_galah_gaia['vr'],df_galah_gaia['vtheta'],df_galah_gaia['vphi'] = cx2sp(x, y, z, vx, vy, vz)
#for vaex data frame
#df_galah_gaia.r,df_galah_gaia.theta,df_galah_gaia.phi,df_galah_gaia.vr,df_galah_gaia.vtheta,df_galah_gaia.vphi = cx2sp(x, y, z, vx, vy, vz)

In [ ]:
df_galah_gaia[:3]

In [ ]:
r , theta, phi, vr, vtheta, vphi = df_galah_gaia['r'],df_galah_gaia['theta'],df_galah_gaia['phi'],df_galah_gaia['vr'],df_galah_gaia['vtheta'],df_galah_gaia['vphi'] 

In [ ]:
# #A routine to make scattter plot in mollweide
# def plot_molleweide(gl,gb,color_data,color_label,title):
#   gl[gl>180]-=360
#   plt.figure(figsize=(20,10))
#   ax=plt.subplot(111,projection='mollweide')
#   im=ax.scatter(np.radians(gl),np.radians(gb),alpha=1,s=7,c=color_data)
#   plt.title('%s(%dstars)'%(title,len(gl)))
#   plt.colorbar(im,label=color_label)
#   plt.show()
#   plt.clf()
#   plt.close()

In [ ]:
# plot_molleweide(df_galah_gaia['l'],df_galah_gaia['b'],df_galah_gaia['b'],'Fe_h[dex]','Kgiants')

In [ ]:
import matplotlib.pyplot as plt 
plt.plot(df_galah_gaia['vr'], df_galah_gaia['vphi'], '.')
plt.xlabel(r'$ v_r \,(kpc)$')
plt.ylabel(r'$v_{\phi} \,(kpc)$')

In [ ]:
df_galah_gaia['Vtot']=np.sqrt(df_galah_gaia['vr']*df_galah_gaia['vr']+(df_galah_gaia['vphi']-232.)*(df_galah_gaia['vphi']-232.)+df_galah_gaia['vz']*df_galah_gaia['vz'])
dfhalo=df_galah_gaia[abs(df_galah_gaia['Vtot'])>210.].copy()
print(len(dfhalo))
o = dfhalo['vphi']
print(o)

In [ ]:
import galpy
#from galpy.potential import MWPotential2014 as pot
from galpy.potential.mwpotentials import McMillan17 as pot
from galpy.util.bovy_conversion import get_physical
from galpy.actionAngle import actionAngleStaeckel
from galpy.orbit import Orbit


In [ ]:
np.mean(dfhalo['vphi'])

In [ ]:
from astropy.stats import median_absolute_deviation

In [ ]:
mad = median_absolute_deviation(dfhalo['vphi'])
mad

In [ ]:
###velocity vs  metallicity
plt.scatter(dfhalo['fe_h'],dfhalo['vr'],s=0.9)
plt.xlabel('Fe/H')
plt.ylabel('vr')
plt.show()
###velocity vs  metallicity
plt.scatter(dfhalo['fe_h'],dfhalo['vtheta'],s=0.9)
plt.xlabel('Fe/H')
plt.ylabel('vtheta')
plt.show()
###velocity vs  metallicity
plt.scatter(dfhalo['fe_h'],dfhalo['vphi'],s=0.9)
plt.xlabel('Fe/H')
plt.ylabel('vphi')
plt.show()

In [ ]:
vphi = np.mean(dfhalo['vphi'])

In [ ]:
dfhalo['vphi'].min()

In [ ]:
dfhalo['vphi'].max()

In [ ]:
vphi.min()

In [ ]:
dfhaloo = vaex.from_astropy_table(dfhalo)

In [ ]:
dfhaloo.export_hdf5('/content/dfhalo.hdf5')

In [ ]:
z = dfhaloo.fe_h
# Lz = dfhaloo.Lz
# E = = dfhaloo.E

In [ ]:
dfhaloo.plot("Lz", "E",
        limits='99.7%',
        z="fe_h:-2.5,-1,5", show=True, visual=dict(row="z"),
        figsize=(12,8), f="log", wrap_columns=2);

In [ ]:
#calculate the value of beta
def beta(rsig, tsig, psig):
  return 1.0 - 0.5*(tsig**2 + psig**2)/(rsig**2) 

In [ ]:
beta(2,1.5,2.0)

In [ ]:
#df=df[df['r']<9.5]
R_gc=np.arange(dfhalo['r'].min(),dfhalo['r']. max(),4)
len(R_gc)

In [ ]:
dfhalo['r'].max()

In [ ]:
dfhalo 

#### Here we use |z|> 5. 

In [ ]:
z_neg = dfhalo['z']<-5
z_pos = dfhalo['z']> 5 

In [ ]:
z = (~z_neg) & (~z_pos)

In [ ]:
dfhalo_ =dfhalo[~z]

In [ ]:
plt.plot(dfhalo_['x'], dfhalo_['z'], '.')
plt.xlabel(r'$x$')
plt.ylabel(r'$z$')


In [ ]:
#KGs
df = dfhalo_
plt.rcParams['figure.figsize'] = 10, 5
rspacing = np.arange(4, 124, 5)
rmin, rmax = 0, 125.
z_range = [0.,50]
halokg = []
def kgiant(rmin, rmax, z_range, datatype=None):
  kgs = df[((df['r']>rmin) & (df['r']<rmax) &
                   (np.abs(df['z'])>z_range[0]) & (np.abs(df['z'])<z_range[1]))]
  halokg.append(kgs)
  return kgs

kg = kgiant(rmin, rmax, z_range)

plt.hist(kg[kg['fe_h']>-1.4]['r'], histtype='step', bins=rspacing, density= True,
         lw=3, color='darkblue', ls='--',
         label='$\mathsf{KGs}:\ [\mathsf{Fe/H}]> -1.4\ (N=%d)$'%kg[kg['fe_h']>-1.4]['r'].shape[0] )
plt.hist(kg[kg['fe_h']<=-1.4]['r'], histtype='step', bins=rspacing, density= True,
         lw=3, color='r', ls='--', 
         label='$\mathsf{KGs}:\ [\mathsf{Fe/H}]\leq -1.4\ (N=%d)$'%kg[kg['fe_h']<=-1.4]['r'].shape[0])

plt.legend(loc='upper left', fontsize=18)


plt.xlabel('$\mathsf{galacto-centric\ radius}\ (r/\mathsf{kpc})$', fontsize=22)
plt.ylabel('$\mathsf{pdf}$', fontsize=22)
plt.minorticks_on()




In [ ]:
plt.scatter(halokg[0]['x'], halokg[0]['z'])
plt.xlabel(r'$x$')
plt.ylabel(r'$z$')
plt.savefig('plot_zvsx.png')

In [ ]:
plt.plot(halokg[0]['x'], halokg[0]['y'], '.')
plt.xlabel(r'$x$')
plt.ylabel(r'$z$')
plt.savefig('plot_yvsx.png')

In [ ]:
vr =halokg[0]['vr']
vphi =halokg[0]['vphi']
vtheta = halokg[0]['vtheta']


In [ ]:
#df=df[df['r']<9.5]
R_gc=np.arange(halokg[0]['r'].min(),halokg[0]['r'].max(),4)
len(R_gc)

In [ ]:
def velocity_dispersion(halokg,R_gc):

  rsig=[]
  tsig=[]
  psig=[]
  for i in range(len(R_gc)):
  
    a=[]
    b=[]
    c=[]
    for j in range(len(halokg)):
      if R_gc[i]<halokg[0]['r'][j]<R_gc[i+1]:
        a.append(halokg[0]['vr'][j])
        b.append(halokg[0]['vtheta'][j])
        c.append(halokg[0]['vphi'][j])
    rsig.append(median_absolute_deviation(a))    
    tsig.append(median_absolute_deviation(b))
    psig.append(median_absolute_deviation(c))
  return rsig,tsig,psig

In [ ]:
vr1, vt1, vp1 = vr[0:1050], vtheta[0:150], vphi[0:150]

In [ ]:
np.var(vr1)

In [ ]:
halokg_vaex = vaex.from_astropy_table(halokg[0])

In [ ]:
halokg_vaex

In [ ]:
#halokg_vaex.export("/content/halokg_vaex.csv")
halokg_vaex.export_hdf5("/content/halokg_vaex.hdf5")


In [ ]:
#halokg_vaex_out = vaex.open("/content/halokg_vaex.hdf5")
halokg_astro_out = halokg_vaex_out.to_astropy_table()
halokg_astro_out

In [ ]:
###velocity vs  metallicity
plt.scatter(halokg_astro_out['r'],halokg_astro_out['vr'],s=0.9)
plt.xlabel(r'$r_{gc}$')
plt.ylabel(r'$v_r$')
plt.show()
###velocity vs  metallicity
plt.scatter(halokg_astro_out['r'],halokg_astro_out['vtheta'],s=0.9)
plt.xlabel('$r_{gc}$')
plt.ylabel(r'$v_{\theta}$')
plt.show()
###velocity vs  metallicity
plt.scatter(halokg_astro_out['r'],halokg_astro_out['vphi'],s=0.9)
plt.xlabel(r'$r_{gc}$')
plt.ylabel(r'$v_{\phi}$')
plt.show()


In [ ]:
###velocity vs  metallicity
plt.scatter(halokg_astro_out['fe_h'],halokg_astro_out['vr'],s=0.9)
plt.xlabel(r'$Fe/H$')
plt.ylabel(r'$v_r$')
plt.show()
###velocity vs  metallicity
plt.scatter(halokg_astro_out['fe_h'],halokg_astro_out['vtheta'],s=0.9)
plt.xlabel('$Fe/H$')
plt.ylabel(r'$v_{\theta}$')
plt.show()
###velocity vs  metallicity
plt.scatter(halokg_astro_out['fe_h'],halokg_astro_out['vphi'],s=0.9)
plt.xlabel(r'$Fe/H$')
plt.ylabel(r'$v_{\phi}$')
plt.show()

In [ ]:
dfhalo['r'].max()

In [ ]:
x = np.random.rand(10)
x

In [ ]:
r_gc = dfhalo['r']
r_5 = (r_gc>=5.0) & (r_gc<10.)
r_10 = (r_gc>=10.0) & (r_gc<15.)
r_15 = (r_gc>=15.0) & (r_gc<20.)
r_20 = (r_gc>=20.0) & (r_gc<25.)
r_


In [ ]:
vr_5 = dfhalo[r_5]['vtheta']
np.var(vr_5)

In [ ]:
vr_10 = dfhalo[r_10]['vr']
np.var(vr_10)

In [ ]:
rsig,tsig,psig=velocity_dispersion(halokg,R_gc)


In [ ]:
import galpy
#from galpy.potential import MWPotential2014 as pot
from galpy.potential.mwpotentials import McMillan17 as pot
from galpy.util.bovy_conversion import get_physical
from galpy.actionAngle import actionAngleStaeckel
from galpy.orbit import Orbit

In [ ]:
# !pip install healpy
import healpy as hp 


In [ ]:
#A routine to make scattter plot in mollweide
def plot_molleweide(gl,gb,color_data,color_label,title):
  gl[gl>180]-=360
  plt.figure(figsize=(20,10))
  ax=plt.subplot(111,projection='mollweide')
  im=ax.scatter(np.radians(gl),np.radians(gb),alpha=1,s=7,c=color_data)
  plt.title('%s(%dstars)'%(title,len(gl)))
  plt.colorbar(im,label=color_label)
  plt.show()
  plt.clf()
  plt.close()

In [ ]:
plot_molleweide(halokg[0]['l'],halokg[0]['b'],halokg[0]['fe_h'],'Fe/H[dex]','Kgiants')
plt.savefig('skydristribution.png')
plt.close()

In [ ]:
plt.scatter(halokg[0]['x'], halokg[0]['y'])
plt.xlabel(r'$x$')
plt.ylabel(r'$y$')
plt.close


In [ ]:
plt.scatter(halokg[0]['x'], halokg[0]['z'])
plt.xlabel(r'$x$')
plt.ylabel(r'$z$')

In [ ]:
def plot_mwd(RA,Dec,org=0,title='Mollweide projection', projection='mollweide', ax=None):
    ''' 
    RA, Dec are arrays of the same length.
    RA takes values in [0,360), Dec in [-90,90],
    which represent angles in degrees.
    org is the origin of the plot, 0 or a multiple of 30 degrees in [0,360).
    title is the title of the figure.
    projection is the kind of projection: 'mollweide', 'aitoff', 'hammer', 'lambert'
    '''
    x = np.remainder(RA+360-org, 360) # shift RA values
    ind = x>180
    x[ind] -=360    # scale conversion to [-180, 180]
    x=-x    # reverse the scale: East to the left
    tick_labels = np.array([150, 120, 90, 60, 30, 0, 330, 300, 270, 240, 210])
    tick_labels = np.remainder(tick_labels+360+org, 360)

    if ax==None:
        fig = plt.figure(figsize=(1.5*8, 1.5*4), dpi=100)
        ax = fig.add_subplot(111, projection=projection)
    ax.set_xticklabels(map(lambda x: '$'+np.str(x)+'^\\circ $', tick_labels), fontsize=17)
    ax.get_yticklabels().set_fontsize = 20
    return ax, np.radians(x), np.radians(Dec)

In [ ]:
ax, x, y = plot_mwd(np.degrees(dfhalo['l']), np.degrees(dfhalo['b']), 180, ax=None)
ax.plot(x, y, 'gs', mec='g', markersize=4, label='KG', mfc="None")

ax.set_xlabel('galactic longitude $(l)$', fontsize=22)
ax.set_ylabel('galactic latitude $(b)$', fontsize=22)
ax.grid(alpha=0.5, ls='dashed', lw=1.5)
leg = ax.legend(loc=(0.15, 0.2), frameon=True,
                fancybox=False, shadow=False, fontsize=18)
